In [1]:
import logging
import os
import sys
from io import BytesIO
import pandas as pd
import requests
from datetime import date, timedelta, datetime
from collections import Counter
import numpy as np
from dotenv import load_dotenv
from tns_api import get_object
import json
from astroquery.vizier import Vizier
import astropy.units as u
from astropy.coordinates import SkyCoord
from tqdm import tqdm
import time

In [2]:
load_dotenv()

True

In [3]:
TNS_API_KEY= os.getenv("api_key")  # Works on real and sandbox now
TNS_BOT_NAME = os.getenv("name")
TNS_BOT_ID = os.getenv("tns_id")

In [4]:
CATALOG_URL = 'https://www.wis-tns.org/system/files/tns_public_objects/tns_public_objects.csv.zip'

In [5]:
def download_table():
    with requests.post(
        CATALOG_URL,
        headers={
            'user-agent': 'tns_marker{{"tns_id":"{id}","type": "bot", "name":"{name}"}}'.format(
                id=TNS_BOT_ID,
                name=TNS_BOT_NAME,
            )
        },
        data={
            'api_key': (None, TNS_API_KEY)
        },
    ) as response:
        return response.content

In [6]:
data = download_table()

In [7]:
df = pd.read_csv(
        BytesIO(data),
        skiprows=1,
        compression='zip',
    )

In [8]:
date.today()

datetime.date(2024, 11, 27)

In [9]:
df["creationdate"] = pd.to_datetime(df["creationdate"])
df["lastmodified"] = pd.to_datetime(df["lastmodified"])

In [10]:
now = datetime.now()
last_week = now - timedelta(days=7)

In [11]:
last_week_rows = df[df['lastmodified'] > last_week]

In [12]:
last_week_rows.columns

Index(['objid', 'name_prefix', 'name', 'ra', 'declination', 'redshift',
       'typeid', 'type', 'reporting_groupid', 'reporting_group',
       'source_groupid', 'source_group', 'discoverydate', 'discoverymag',
       'discmagfilter', 'filter', 'reporters', 'time_received',
       'internal_names', 'Discovery_ADS_bibcode', 'Class_ADS_bibcodes',
       'creationdate', 'lastmodified'],
      dtype='object')

In [13]:
last_week_visible = last_week_rows.iloc[np.where(last_week_rows["declination"] > -30)]

In [226]:
def is_close(distance, brightness):
    print(distance)
    print(brightness)
    print((10 / (brightness - 3)))
    print(np.log10(distance))
    return ((10 / (brightness - 3)) > np.log10(distance))

In [25]:
def get_bright_star(sky_object, cone = 5*u.arcsec):
    result = Vizier.query_region(sky_object,
                             radius=cone,
                             catalog='I/355/gaiadr3',
                             column_filters={'Gmag': '<15'})
    if len(result) == 0:
        return False
    return len(result[0]) > 0

In [26]:
too_close = []
for i in tqdm(SkyCoord(last_week_visible["ra"], last_week_visible["declination"], frame = "icrs", equinox = 'J2000', unit="deg")):
    too_close.append(get_bright_star(i))
    time.sleep(2)

100%|██████████| 553/553 [23:01<00:00,  2.50s/it]


In [27]:
too_close = np.asarray(too_close)

In [31]:
last_week_visible_seperate = last_week_visible.iloc[np.where(too_close == False)]

In [32]:
Counter(last_week_visible_seperate["reporting_group"])

Counter({'Pan-STARRS': 98,
         'ZTF': 183,
         'YSE': 28,
         'XOSS': 13,
         'ATLAS': 56,
         'Fink': 1,
         'ALeRCE': 78,
         'SGLF': 4,
         'GOTO': 19,
         'BlackGEM': 7,
         'GaiaAlerts': 56,
         'MASTER': 3,
         nan: 1,
         'PALEO': 1})

In [33]:
last_week_visible_seperate.iloc[np.where(last_week_visible_seperate["reporting_group"] == "ZTF")]["internal_names"]

9      ZTF24abfsxul, ATLAS24obp, GOTO24ftk, PS24lno
10                ZTF24abbxukc, ATLAS24nqh, PS24lnz
60                                     ZTF24abudivs
82                                     ZTF24abuhfmt
83                                     ZTF24abttsko
                           ...                     
673               ZTF24absgzvv, PS24kya, ATLAS24qgi
693                                    ZTF24abtdiyv
719             ZTF24absoxlw, ATLAS24qha, GOTO24ijv
721                        ZTF24abrpqhh, ATLAS24qes
722                        ZTF24abrophb, ATLAS24qbp
Name: internal_names, Length: 183, dtype: object

In [34]:
last_week_visible_seperate

,objid,name_prefix,name,ra,declination,redshift,typeid,type,reporting_groupid,reporting_group,...,discoverymag,discmagfilter,filter,reporters,time_received,internal_names,Discovery_ADS_bibcode,Class_ADS_bibcodes,creationdate,lastmodified
0,167232,AT,2024acjn,41.849622,5.635647,NaN,NaN,NaN,4.0,Pan-STARRS,...,20.7100,26.0,w,"K. C. Chambers, T. de Boer, J. Herman, J. Fair...",2024-11-27 00:03:37,PS24loa,NaN,NaN,2024-11-27 00:03:38,2024-11-27 00:03:38
1,167231,AT,2024acjm,82.145227,5.134683,NaN,NaN,NaN,4.0,Pan-STARRS,...,20.6100,26.0,w,"K. C. Chambers, T. de Boer, J. Herman, J. Fair...",2024-11-27 00:03:04,PS24lny,NaN,NaN,2024-11-27 00:03:05,2024-11-27 00:03:05
2,167230,AT,2024acjl,338.812776,0.355358,NaN,NaN,NaN,4.0,Pan-STARRS,...,20.7500,26.0,w,"K. C. Chambers, T. de Boer, J. Herman, J. Fair...",2024-11-27 00:02:48,PS24lnx,NaN,NaN,2024-11-27 00:02:49,2024-11-27 00:02:49
3,167229,AT,2024acjk,10.069270,4.575079,NaN,NaN,NaN,4.0,Pan-STARRS,...,20.7300,26.0,w,"K. C. Chambers, T. de Boer, J. Herman, J. Fair...",2024-11-27 00:02:36,PS24lnw,NaN,NaN,2024-11-27 00:02:37,2024-11-27 00:02:37
4,167228,AT,2024acjj,61.570253,-9.886095,NaN,NaN,NaN,4.0,Pan-STARRS,...,19.5400,26.0,w,"K. C. Chambers, T. de Boer, J. Herman, J. Fair...",2024-11-27 00:02:10,PS24lnv,NaN,NaN,2024-11-27 00:02:11,2024-11-27 00:02:11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,166311,SN,2024abbj,51.970355,17.651277,0.058000,3.0,SN Ia,48.0,ZTF,...,20.2929,110.0,g,"Jesper Sollerman (SU), Christoffer Fremling (C...",2024-11-14 05:15:41,"ZTF24absoxlw, ATLAS24qha, GOTO24ijv",2024TNSTR4497....1S,2024TNSCR4536....1F,2024-11-14 05:15:42,2024-11-20 20:31:21
720,166179,SN,2024aawh,202.366235,11.006286,0.022799,10.0,SN II,10.0,XOSS,...,18.1100,1.0,Clear,"Jianlin Xu, Mi Zhang, Leiming Tang, Yanxi Li, ...",2024-11-11 02:35:03,"PSP24aa, ZTF24abrirtr",2024TNSTR4438....1X,2024TNSCR4561....1S,2024-11-11 02:35:04,2024-11-20 13:50:35
721,166144,AT,2024aauy,298.080591,47.240900,NaN,NaN,NaN,48.0,ZTF,...,19.8604,110.0,g,"Jesper Sollerman (SU), Christoffer Fremling (C...",2024-11-10 07:37:21,"ZTF24abrpqhh, ATLAS24qes",2024TNSTR4423....1S,NaN,2024-11-10 07:37:22,2024-11-20 20:31:20
722,166095,SN,2024aatb,165.264175,57.786534,0.047905,3.0,SN Ia,48.0,ZTF,...,20.2117,111.0,r,"Jesper Sollerman (SU), Christoffer Fremling (C...",2024-11-09 11:39:34,"ZTF24abrophb, ATLAS24qbp",2024TNSTR4406....1S,2024TNSCR4561....1S,2024-11-09 11:39:36,2024-11-20 12:21:25


In [36]:
last_week_visible_seperate.to_csv("ToO_Objects-20241127.csv", index = False)